<a href="https://colab.research.google.com/github/joelfreeman38/project-2-team-6/blob/master/Copy_of_cg_arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analysis - Housing Price Prediction using Zillow Data

In [4]:
# Importing what we need
import numpy as np
e = np.e
import pandas as pd
import matplotlib
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

import warnings
warnings.filterwarnings('ignore')

## Read the dataset

In [3]:
# Read the data
df = pd.read_csv("../resources/data/metro_zillow.csv", nrows=1000)

# Taking a look at the shape of the dataset
print(df.shape)

# This line will tell us how many of our rows represent metro areas
print(df.RegionType.value_counts())

# Examining the first five rows of our dataframe
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../resources/data/metro_zillow.csv'

The raw dataframe contains 928 rows and 87 columns. Among these columns, five provide details about the metro area associated with each row. Columns 6 through 82 represent a time series spanning from March 2018 to December 2024, with values indicating the median home price for that metro area in each specific month.

Notably, out of the 928 rows, the last row is unique in that it does not correspond to a specific metro area. Instead, it tracks the median home price for the entire United States.

## Investing in California
* For an investment in California, we need to filter down our dataset to metro areas in CA

In [ ]:
# Making a dataframe of just California data
df_ca = df[df['StateName'] == 'CA']

# Seeing how many rows we get
print(df_ca.shape)

# Sanity check
df_ca.head()

* So after filtering our dataframe down, we are left with 34 metro areas throughout California.

## Checking for NaN values

In [ ]:
# Checking our dataframe for NaN values
print(f'There are {df_ca.isna().sum().sum()} NaNs in our original dataframe')

# Backfilling that single NaN
df_ca.fillna(method='bfill', inplace=True)

# Sanity check
print(f'There are {df_ca.isna().sum().sum()} NaNs after using backfill')

In [ ]:
#Price distribution within Michigan
# Getting a list of the values for the last date in our time series
current_median_msa_home_prices = list(df_ca['12/31/2024'])

# Plotting the results
fig, ax = plt.subplots(figsize=(10,5))
plt.hist(current_median_msa_home_prices, bins=20)
plt.title('2024 CA State Median Home Price by Metro Area')
plt.xlabel('Median Home Price')
plt.ylabel('Count')
plt.show()

* We have a few outliers, some of which are Santa Maria and Salinas area, as confirmed below. However, this doesn't necessarily imply that they are the best or worst investment choice, despite what our models may suggest.

In [ ]:
# Checking the median home price for Ann Arbor in the most recent month.
int(df_ca[df_ca['RegionName'] == 'Salinas, CA']['12/31/2024'])

## Melt Data Function

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.
    Returns a long-form datetime dataframe
    with the datetime column names as the index and the values as the 'values' column.

    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    melted = pd.melt(df, id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

## Time Series for California

In [ ]:
# Melting the California dataframe
df_ca_melted = melt_data(df_ca)

# Plotting the average time series for all of CA state
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(df_ca_melted)
plt.title('Averaged Median Home Prices in CA State from 2018 - 2025')
plt.xlabel('Year')
plt.ylabel('Averaged Median Home Price')

## Reshaping the Dataframe

In [ ]:
# Reshaping the dataframe
df_reshaped = pd.DataFrame()
for i in df_ca['RegionName']:
    x = melt_data(df_ca[df_ca['RegionName'] == i])
    df_reshaped = pd.concat([df_reshaped, x], axis=1)
    df_reshaped.rename(columns = {'value':i}, inplace = True)

df_reshaped.head()

# Time Series Length
* A key characteristic of our raw dataset must be addressed: Like the rest of the U.S., California experienced a sharp decline in home sales in 2021. If we train our models on the entire dataset, they may be disproportionately influenced pre-2020, sub 3% interest rates, which could impact their predictive accuracy—especially since our business use case focuses on forecasting just one year ahead.

* To prevent this from compromising model performance in validation tests and future predictions, we will limit our training data to only include data from March 2018 onward.

In [ ]:
# Reshape to make sure we train our model from Mar 2018 and beyond
df_2018 = df_reshaped['2018-03-31':]

# Examining the new shape
print(df_2018.shape)

# Sanity check
df_2018.head()

## Train Test Split

* To validate our models and assess their effectiveness, we need to split our time series data into a training set and a test set. In a time series context, the training set will consist of the first 80% of the data, while the test set will cover the remaining 20%. Since our post-March 2018 data spans 6.92 years, this results in a training period of 5.25 years and a test period of 1.67 years.

* This training-to-test ratio strengthens the justification for our predictions. If our models can accurately forecast values 2 years ahead based on 6.92 years of data, we can be more confident in their ability to predict values one year ahead using a larger dataset.

In [ ]:
# Printing out the lengths of our unsplit time series
print(f'Whole series lengths: {len(df_2018)} \n')

# Manually dividing the data into train and test sets
train = df_2018[:'2023-05-31']
test = df_2018['2023-05-31':]

# Printing the lengths of our new train and test sets
print(f'Train set lengths: {len(train)}')
print(f'Test set lengths: {len(test)} \n')

# Checking that the proportions are how we want them
print(f'Train proportion = {round(len(train) / len(df_2018),1)}')
print(f'Test proportion = {round(len(test) / len(df_2018),1)} \n')

# Checking the length in years of our train and test sets
print(f'Train set length in years: {round(len(train) / 12, 2)}')
print(f'Test set length in years: {round(len(test) / 12, 2)}')

## Setting a Performance Benchmark Before Model Training
* To assess whether modeling will provide value in this business context, we will compare our models' performance against a simple, exploratory data analysis approach.

In [ ]:
# Establishing a Performance Baseline Before Modeling
# These lists will hold the names of each metro area, as well as that area's ROI from 2012 to 2021
names = []
historical_roi = []

# This for loop adds the information to the two lists
for i in range(len(train.columns)):

    clean_name = train.columns[i][:-4]

    initial_val = train[train.columns[i]]['2018-03-31']
    present_val = train[train.columns[i]]['2023-04-30']

    roi = round(((present_val - initial_val) / initial_val) * 100, 2)

    names.append(clean_name)
    historical_roi.append(roi)

# Turning the data into a pandas dataframe
roi_df = pd.DataFrame()
roi_df['City'] = names
roi_df['% ROI'] = historical_roi
roi_df.sort_values(['% ROI'], inplace=True, ascending=False)
roi_df.set_index('City', inplace=True)

# Plotting the historical data
fig, ax = plt.subplots(figsize=(20,5))
plt.bar(roi_df.index, roi_df['% ROI'])
plt.title('% ROI by Metro Area 2018 - 2023')
plt.xlabel('Metro Area')
plt.ylabel('% ROI')
plt.xticks(rotation=45)
plt.axhline(0, color='k')
plt.show()

# Displaying our top five choices based on EDA
roi_df.head()

In [ ]:
# Our top five choices based on EDA results
top_5 = ['Visalia, CA', 'Yuba City, CA', 'Sonora, CA', 'Crescent City, CA', 'Hanford, CA']

# These two lists will track our buy and sell numbers
buys = []
sells = []

# Getting the median values for 2023 (buys) and 2024 (sells)
for i in top_5:
    buys.append(df_2018[i]['2023-04-30'])
    sells.append(df_2018[i]['2024-12-31'])

# Calculating the ROI we would have achieved
eda_roi = round( ((sum(sells) - sum(buys)) / sum(buys) ) * 100, 2)

# Printing the ROI
print(f'Using an EDA approach, we could have achieved {eda_roi}% ROI from 2023 to 2024')

## Modeling
* Preparing the functions - Since our goal is to identify the five most optimal metro areas for investment within California, we need to run time series models for each metro area individually. Given that there are 34 metro areas, doing this manually is impractical, so we are automating the process by writing functions to handle these steps efficiently.

In [ ]:
def log_transform(series_i):

    '''Takes in a series and returns the log transformed version of that series'''

    log_transformed = np.log(series_i)
    dropped_nans = log_transformed.dropna()
    return dropped_nans

In [ ]:
def run_auto_arima(series_i):

    '''ARIMA (Autoregressive Integrated Moving Average) is specifically designed to model time
    series data, meaning it analyzes patterns within a sequence of data points ordered by time,
    allowing prediction of future values based on past trends.
    Runs a grid search on the series passed in, then instantiates and fits
    an ARIMA model with those hyperparameters, then returns that fit model. '''
    # Function to automatically select and fit SARIMA model
def run_auto_sarima(series_i):
    """Runs a grid search for the best SARIMA model parameters and fits the SARIMA model."""
    gridsearch = auto_arima(
        series_i,
        start_p=0, max_p=3,
        d=None, max_d=3,
        start_q=0, max_q=3,
        seasonal=True,
        m=12,  # Monthly data seasonality
        start_P=0, max_P=3,
        start_Q=0, max_Q=3,
        max_order=10,
        stepwise=True,
        suppress_warnings=True
    )

    model = SARIMAX(
        series_i,
        order=gridsearch.order,
        seasonal_order=gridsearch.seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    return model.fit()


In [ ]:
def run_sarima_model(i, steps, df):
    """Runs SARIMA on the selected time series, returning forecast results."""
    series = df.iloc[:, i:i+1]
    name = series.columns[0]
    log_series = np.log(series).dropna()

    model = run_auto_sarima(log_series)
    log_forecast = model.get_forecast(steps)
    forecast_series = np.exp(log_forecast.summary_frame())

    return name, series, forecast_series

In [ ]:
# Function to plot results
def plot_sarima_results(i, steps, df):
    name, original_series, forecast_series = run_sarima_model(i, steps, df)

    plt.figure(figsize=(15, 7))
    plt.plot(original_series, label='Original')
    plt.plot(forecast_series['mean'], label='Predicted')
    plt.fill_between(
        forecast_series.index,
        forecast_series['mean_ci_lower'],
        forecast_series['mean_ci_upper'],
        color='gray', alpha=0.2
    )
    plt.title(name)
    plt.legend()
    plt.xlabel('Year')
    plt.ylabel('Median Home Price')
    plt.show()

    forecast = round(forecast_series['mean'][steps - 1])
    low_int = round(forecast_series['mean_ci_lower'][steps - 1])
    high_int = round(forecast_series['mean_ci_upper'][steps - 1])
    print(f"{steps}-month forecast: {forecast}")
    print(f"95% confidence interval: {low_int} - {high_int}")

# Example usage (Make sure df_2018 is loaded before running this)
# plot_sarima_results(1, 12, df_2018)


In [ ]:
def evaluate_models(df1, df2):

    '''This function takes in two dataframes (train and test in our case),
    and returns a dataframe with how accurate the models fit to the train
    set were in predicting the test set values.'''

    names = []
    actuals = []
    preds = []
    perc_errors = []

    for i in range(len(train.columns)):

        name, series, forecast_series = run_arima_model(i, 24, df1)

        clean_name = name[:-4]

        actual_val = df2[name][-1]
        predicted_val = forecast_series.iloc[23, 0]
        error = abs(actual_val - predicted_val)
        percent_error = (error/ actual_val) * 100

        names.append(clean_name)
        actuals.append(f'{round(actual_val):,}')
        preds.append(f'{round(predicted_val):,}')
        perc_errors.append(round(percent_error, 2))

        #print(train.columns[i][:-4], 'done', f'{i+1}/26')


    results_df = pd.DataFrame(index=names)
    results_df['2024 Actual'] = actuals
    results_df['2024 Predicted'] = preds
    results_df['% Error'] = perc_errors
    results_df.sort_values(by='% Error', inplace=True)

    return results_df

In [ ]:
def generate_predictions(df, steps):

    '''Similar to evaluate_models(), this function takes in a dataframe,
    and a specific number of steps, and returns a dataframe of the
    future predictions the specified number of steps past the end of
    the sample.'''

    names = []
    current_vals = []
    pred_vals = []
    net_profits = []
    ROI_strings = []

    count = 0
    for i in range(len(df.columns)):

        count += 1

        name, series, forecast = run_arima_model(i, steps, df)

        clean_name = name[:-4]
        print(clean_name)

        cur_val = series.iloc[-1, 0]
        pred_val = forecast.iloc[steps-1, 0]
        net_prof = round(pred_val - cur_val , 2)
        roi = int(round(((pred_val - cur_val) / cur_val) * 100, 2))

        names.append(clean_name)
        current_vals.append(f'{round(cur_val):,}')
        pred_vals.append(f'{round(pred_val):,}')
        net_profits.append(f'{round(net_prof):,}')
        ROI_strings.append(f'{roi}%')

        if count == 26:
            break


    results_df = pd.DataFrame()
    results_df['City'] = names
    results_df.set_index(['City'])
    results_df['Current Value'] = current_vals
    results_df['Predicted Value'] = pred_vals
    results_df['Net Profit'] = net_profits
    results_df['ROI'] = ROI_strings

    return results_df

## Model Evaluation
* To ensure our future predictions remain reasonably accurate, we will first assess how well our models perform on existing data. The evaluate_models() function will measure the difference between our model predictions for December 2024 and the actual observed values for that month.

* Since the models are trained only on data from 2018 to 2023, their predictions for December 2024 are made without any knowledge of the test data, which spans from 2023 to 2024. This approach helps validate their reliability in forecasting future trends.

In [ ]:
# This dataframe will show us how accurate our models are
eval_df = evaluate_models(train, test)

# Displaying the dataframe
eval_df

In [ ]:
eval_df = eval_df.iloc[:-1]
eval_df

In [ ]:
# Calculating the average error using the dataframe above
average_error = str(round(sum([int(i) for i in eval_df['% Error']]) / len(eval_df) , 2)) + '%'

# Printing the result
print(f"On average our model based predictions were {average_error} off from the observed values.")

* On average, our model's predictions were about 9.21% off from the actual values in our test set. We are satisfied with these validation results, especially since we haven’t manually tuned any of the models.

* Moreover, this 9.21% average error comes from models trained on 5 years of data to predict two years ahead. In our final recommendations, we will use ten years of training data to predict just one year ahead. This suggests that our future predictions will likely be even more accurate than those in this validation test.

In [ ]:
# Getting predictions from our models for December of 2024
model_predictions_2024 = generate_predictions(train, 24)

# Checking out the results
model_predictions_2024

In [ ]:
# A list of the top five metro areas output by our models
model_top_5 = ['Madera, CA', 'Santa Cruz, CA', 'Merced, CA', 'Hanford, CA', 'Stockton, CA']

# Variables to track initial and final investment value
buys = 0
sells = 0

# Adding the relevant values
for i in model_top_5:
    buys  += df_2018[i]['2023-04-30']
    sells += df_2018[i]['2024-12-31']

# Calculating the ROI
model_roi_2023 = round(((sells - buys) / buys) * 100 , 2)

# Displaying the results
print(f'Using modeling, we could have achieved {model_roi_2023}% ROI from 2023 to 2024')

* Switching from an EDA-based approach to modeling would have increased ROI from -0.26 % to -0.18 % between April 2023 and December 2024. This confirms that our modeling efforts are worthwhile and significantly impact this specific business outcome.

## Predictions
* The goal of this project is to give potential clients a starting point for investing in California real estate by identifying the five metro areas best suited for short-term investment. We have now reached the stage where we can make these predictions.

* Using the same modeling process as before, we will apply our models to the full 2018–2024 dataset to forecast the median home value for each of the 34 metro areas in December 2025. We will then rank these 26 growth estimates by highest projected ROI and select the top five as our final recommendations.

In [ ]:
# Generating predictions one year past our entire 2018 to 2024 dataset
recommendation_df = generate_predictions(df_2018, 12)

# Looking at the results
recommendation_df.iloc[1:,:]

In [ ]:
plot_results(1, 12, df_2018)

In [ ]:
plot_results(2, 12, df_2018)

In [ ]:
plot_results(4, 12, df_2018)

In [ ]:
plot_results(6, 12, df_2018)

In [ ]:
plot_results(9, 12, df_2018)